In [2]:
import sys
dll_path = r'C:\Users\st087829\PycharmProjects\some_cybernetic_scheisse\dobot_lib'
sys.path.insert(1,dll_path)
import DobotDllType as dType
from math import pi
import numpy as np
import time

In [4]:
com = 'COM4'
api = dType.load(dll_path)

The dll you are using is 64-bit. In order to run smoothly, please ensure that your python environment is also 64-bit.
The python environment is： ('64bit', 'WindowsPE')


In [7]:
state = dType.ConnectDobot(api, "", 115200)[0]
print("Connect status:", state)
#if (state == dType.DobotConnect.DobotConnect_NoError):
    #Dobot interactive codes
#dType.DisconnectDobot(api)

Connect status: 0


In [6]:
dType.DisconnectDobot(api)

In [8]:
dType.SetQueuedCmdClear(api)
dType.SetQueuedCmdStartExec(api)

In [9]:
dType.SetHOMEParams(api, 200, 200, 200, 200, isQueued = 1)
dType.SetPTPJointParams(api, 200, 200, 200, 200, 200, 200, 200, 200, isQueued = 1)
dType.SetPTPCommonParams(api, 100, 100, isQueued = 1)

[4]

In [30]:
def move_xy_pixel(xy, r_ab, o_o1, l, j4_rot=-25+45, z=10):
    pixel_tile_size = 45
    pixel_center = np.array([1280//2, 360-pixel_tile_size//2])
    xy_on_board = (np.array(xy) - pixel_center)*np.array([-1, 
                                                          1])/pixel_tile_size + np.array([3.5, 3.5])
    print(xy_on_board)
    move_to_tile(xy_on_board, r_ab, o_o1, l, j4_rot, z)
    

In [11]:
def move_to_tile(xy_tile, r_ab, o_o1, l, j4_rot=-25+45, z=10):
    xy_tile = np.array(xy_tile)
    robot_coords = r_ab@xy_tile*l + o_o1
    
    dType.SetPTPCmd(api,
        dType.PTPMode.PTPMOVLXYZMode,
        robot_coords[0],
        robot_coords[1],
        z,
        j4_rot,
        isQueued=1
    )

In [48]:
move_xy_pixel([662, 359], r_ab, o_o1, r)

[3.01111111 3.96666667]


In [42]:
move_to_tile([0, 0], r_ab, o_o1, r, z=-70)

In [219]:
dType.GetPose(api)[2]

-7.301727294921875

In [217]:
dType.SetEndEffectorGripper(api, 1, 1, isQueued=1)

[175]

In [57]:
from math import dist
from itertools import combinations, islice

def coords_from_chess_pos(pos):
    # from chess pos (e.g. 'e4') into local chess coords
    # letters - xi axis, numbers - eta. Origin at center of a1
    assert len(pos) == 2
    char, num = pos.lower()
    xi = ord(char) - ord('a')
    eta = int(num) - 1
    return xi, eta

def find_intersections(o1, r1, o2, r2, eps=.1):
    # finds intersection points of 2 circles
    # if no intersection, raises exception
    o1, o2 = numpify(o1, o2)
    d = dist(o1, o2)
    assert d <= r1+r2
    assert d > eps
    a = (r1**2-r2**2+d**2)/(2*d)
    h = (r1**2 - a**2)**.5
    P = (o2-o1)*a/d+o1
    perp_ = perp(o2-o1)
    I1, I2 = P+h*perp_, P-h*perp_
    return I1, I2

def norm(vec):
    return np.array(vec)/length(vec)

def perp(vec):
    return norm(vec)[::-1]*np.array([-1,1])

def length(vec):
    return dist(vec, [0]*len(vec))

def numpify(*args):
    arrays = [0]*len(args)
    for i, arg in enumerate(args):
        arrays[i] = arg if isinstance(arg, np.ndarray) else np.array(arg)
    return arrays

def find_circles(A_b, B_b, A_d, B_d):
    # find center and radius of circle, given coords in board frame and dobot frame
    # returns o1, o2, r
    A_b, B_b, A_d, B_d = numpify(A_b, B_b, A_d, B_d)
    a = length(A_d)
    b = length(B_d)
    c = length(A_d - B_d)
    cos_alpha = (a**2+b**2-c**2)/2/a/b
    sin_alpha = (1-cos_alpha**2)**.5
    AB_b = B_b-A_b
    l = length(AB_b)
    r = l/2/sin_alpha
    normal = r*cos_alpha*perp(AB_b)
    M = (A_b+B_b)/2
    o1, o2 = M+normal, M-normal
    return o1, o2, r

def count_coincides(A, points, eps = .1):
    return len([0 for point in points if dist(A, point) < eps])

def is_clockwise(vec1, vec2):
    # finds if vec2 is rotated clockwise from vec1
    cross = vec1[0]*vec2[1] - vec1[1]*vec2[0]
    assert cross != 0
    return cross < 0

def find_center(board_coords, dobot_coords):
    # find center, given coords of 3 points
    # points go clockwise, relative to dobot and all have different angle for 1st joint
    assert len(board_coords) == len(dobot_coords) == 3
    circles = []
    board_coords = list(map(coords_from_chess_pos, board_coords))
    for i, j in combinations([0,1,2], 2):
        o1, o2, r = find_circles(board_coords[i], board_coords[j], dobot_coords[i], dobot_coords[j])
        circles.extend(([o1, r], [o2, r]))
    inters = []
    for circ1, circ2 in combinations(circles, 2):
        inters.extend(find_intersections(*circ1, *circ2))
    centers = [p for p in inters if count_coincides(p, inters) == 3]
    c0 = centers[0]
    eps = .1
    for c in centers:
        if dist(c0, c) > eps:
            c1 = c
            break
    if is_clockwise(board_coords[0] - c0, board_coords[1] - c0):
        return c0
    else:
        return c1

In [58]:
find_center(['b2', 'd3', 'b5'],[[5, 1], [3, 0], [5, -2]])

array([6., 2.])

In [4]:
a = np.array([1,2])

In [30]:
list(itertools.islice([1,2,3], 1, None))

[2, 3]

In [6]:
isinstance(a, np.ndarray)

True

In [5]:
type(a)

numpy.ndarray

In [12]:
delta_h = 50
def pick_up():
    dType.SetPTPCmd(api,
        dType.PTPMode.PTPMOVLXYZMode,
        *dType.GetPose(api)[:2],
        dType.GetPose(api)[2]-delta_h,
        dType.GetPose(api)[3],
        isQueued=1
    )
    dType.SetEndEffectorGripper(api, 1, 1, isQueued=1)
    time.sleep(2)
    dType.SetPTPCmd(api,
        dType.PTPMode.PTPMOVLXYZMode,
        *dType.GetPose(api)[:2],
        dType.GetPose(api)[2]+delta_h,
        dType.GetPose(api)[3],
        isQueued=1
    )

def put_down():
    dType.SetPTPCmd(api,
        dType.PTPMode.PTPMOVLXYZMode,
        *dType.GetPose(api)[:2],
        dType.GetPose(api)[2]-delta_h,
        dType.GetPose(api)[3],
        isQueued=1
    )
    dType.SetEndEffectorGripper(api, 1, 0, isQueued=1)
    time.sleep(2)
    dType.SetPTPCmd(api,
        dType.PTPMode.PTPMOVLXYZMode,
        *dType.GetPose(api)[:2],
        dType.GetPose(api)[2]+delta_h,
        dType.GetPose(api)[3],
        isQueued=1
    )

In [223]:
dType.SetPTPCmd(api,
    dType.PTPMode.PTPMOVLXYZMode,
    dType.GetPose(api)[0],
    dType.GetPose(api)[1],
    dType.GetPose(api)[2]-1,
    dType.GetPose(api)[3],
    isQueued=1
)

[180]

In [240]:
dType.SetPTPCmd(api,
    dType.PTPMode.PTPMOVLANGLEMode,
    *dType.GetPose(api)[4:7],
    dType.GetPose(api)[7]-20,
    isQueued=1
)

[197]

In [45]:
def calibrate():
    delta = 4
    y1 = 5
    input('Move to a6 then press Enter')
    p1 = np.array(dType.GetPose(api)[:2])  # A
    input('Move to a2 then press Enter')
    p2 = np.array(dType.GetPose(api)[:2])  # B
    l = sum((p1-p2)**2)**.5/delta
    
    mb, a = (p1-p2)/(delta*l)
    R_ab = np.array([[a,   mb],
                     [-mb, a]])
    o_o1 = p1-R_ab@np.array([0, y1*l])
    return R_ab, o_o1, l

In [46]:
r_ab, o_o1, r = calibrate()

Move to a6 then press Enter
Move to a2 then press Enter


In [47]:
r

26.609117460944162